## 6ª Rodada de questões do CEO

### Bibliotecas

In [1]:
from sqlalchemy import create_engine
import pandas as pd

### Acessando os Bancos de Dados

In [2]:
# Conexão com o Banco
engine = create_engine('sqlite:///db_olist.sqlite')

# Criando a conexão com o Banco
cursor = engine.connect()

**1.** Quantos clientes únicos tiveram seu pedidos com status de “processing”, “shipped” e “delivered”, feitos entre os dias 01 e 31 de Outubro de 2016. Mostrar o resultado somente se o número total de clientes for acima de 5.

In [3]:
query = """
        SELECT 
            o.order_status,
            COUNT(DISTINCT o.order_id) AS total_orders  
        FROM 
            orders o 
        WHERE
            o.order_status IN ('processing', 'shipped', 'delivered')
            AND o.order_purchase_timestamp BETWEEN DATE('2016-10-01') AND  DATE('2016-10-31')
        GROUP BY 
            o.order_status 
        HAVING 
            total_orders > 5
        """

pd.read_sql(query, cursor)

,order_status,total_orders
0,delivered,265
1,shipped,8


**2.** Mostre a quantidade total dos pedidos e o valor total do pagamento, para pagamentos entre 1 e 5 prestações ou um valor de pagamento acima de 5.000,00

In [4]:
query = """
    SELECT 
        op.payment_type, 
        COUNT(op.order_id) AS total_orders,
        SUM(op.payment_value) AS total_pauments_value
    FROM 
        order_payments op 	
    WHERE
        (op.payment_installments BETWEEN 1 AND 5) 
        OR (op.payment_value > 5000)
    GROUP BY 
        op.payment_type
        """

pd.read_sql(query, cursor)

,payment_type,total_orders,total_pauments_value
0,boleto,19784,2869361.27
1,credit_card,60667,7642843.48
2,debit_card,1529,217989.79
3,not_defined,3,0.00
4,voucher,5775,379436.87


**3.** Quantos produtos estão cadastrados nas categorias: perfumaria, brinquedos, esporte lazer e cama mesa, que possuem entre 5 e 10 fotos, um peso que não está entre 1 e 5 g, um altura maior que 10 cm, uma largura maior que 20 cm. Mostra somente as linhas com mais de 10 produtos únicos.



In [5]:
query = """
        SELECT
        p.product_category_name, 
        COUNT(p.product_id) AS quant_products_cadastrado 
    FROM
        products p 
    WHERE
            (p.product_category_name IN ('perfumaria', 'brinquedos', 'esporte_lazer', 'cama_mesa_banho')) 
        AND (p.product_photos_qty BETWEEN 5 AND 10)                                                       
        AND (p.product_weight_g NOT BETWEEN 1 AND 5)                                                      
        AND (p.product_height_cm > 10)
        AND (p.product_width_cm > 20)                                                                     
    GROUP BY 
        p.product_category_name 
    HAVING 
        quant_products_cadastrado > 10
        """

pd.read_sql(query, cursor)

,product_category_name,quant_products_cadastrado
0,brinquedos,68
1,cama_mesa_banho,34
2,esporte_lazer,112


**4.** Qual a quantidade de cidades únicas dos vendedores no estado de São Paulo ou Rio de Janeiro com a latitude maior que -24.54 e longitude menor que -45.63?


In [6]:
query = """
    SELECT
        g.geolocation_state, 
        COUNT(DISTINCT g.geolocation_city) AS quant_unique_cities 
    FROM 
        geolocation g 
    WHERE
        g.geolocation_state IN ('SP', 'RJ')
        AND g.geolocation_lat > -24.54
        AND g.geolocation_lng < -45.63
    GROUP BY g.geolocation_state 
        """

pd.read_sql(query, cursor)

,geolocation_state,quant_unique_cities
0,RJ,9
1,SP,996


**5.** Qual o número de clientes únicos, agrupados por estado e por cidades que comecem com a letra “m”, tem a letra “o” e terminem com a letra “a”? Mostrar os resultados somente para o número de clientes únicos maior que 10.


In [7]:
query = """
    SELECT
        p.product_category_name, 
        COUNT(DISTINCT p.product_id) AS products_cadastrados 
    FROM 
        products p 
    WHERE
            (p.product_category_name LIKE 'a%o')
        AND (p.product_photos_qty > 5)
    GROUP BY
        p.product_category_name 
    HAVING 
        COUNT(DISTINCT p.product_id) > 10
        """

pd.read_sql(query, cursor)

,product_category_name,products_cadastrados
0,automotivo,172


**6.** Quantos produtos estão cadastrados em qualquer categorias que comece com a letra “a” e termine com a letra “o” e que possuem mais de 5 fotos? Mostrar as linhas com mais de 10 produtos



In [8]:
query = """
        SELECT
        c.customer_state, c.customer_city, 
        COUNT(DISTINCT c.customer_id) AS quant_customers 
    FROM 
        customer c 
    WHERE
        c.customer_city LIKE 'm%o%a'
    GROUP BY 
        c.customer_state, c.customer_city 
    HAVING 
        quant_customers > 10
        """

pd.read_sql(query, cursor)

,customer_state,customer_city,quant_customers
0,MG,matias barbosa,13
1,SP,mairipora,75
2,SP,mococa,34
3,SP,mongagua,42
4,SP,monte azul paulista,19
